# **Importing Necessary Libraries**

In [ ]:
import tensorflow as tf
import os
import cv2
import numpy as np 
import sys
import keras
from keras.models import Sequential
from keras.layers import Dense, ConvLSTM2D, Flatten

# **Prediction**

In [ ]:
def load_video(path):
  video = np.zeros((11, resize[1], resize[0], 3))
  cap = cv2.VideoCapture(path)       
  
  if (cap.isOpened()== False):
    sys.exit("Error opening video")
  
  #determine step size 
  noOfFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  step = int(noOfFrames/10)
  if noOfFrames == 0:
    sys.exit("No frames found")
  else:

    count =0 
    while 1:

      #read specific frames
      cap.set(cv2.CAP_PROP_POS_FRAMES, step*count)
      ret, frame =cap.read()
      
      if ret == False or count > 10:   
        break
      
      else:
        frame =cv2.resize(frame, (32,18))
        video[count, :,:,:]=frame/255 #map between 0 and 1 values

        count +=1

In [ ]:
def prediction(test_video, unknown_data=None):
  '''
  Makes predictions and retrun label with its cosine similarity score as list
 
  test_video: str
    path to the testing video. This must be a single instance of an adverstisement. It must be in format '.avi'

  unknown_data: list
    it must be of size of 2 with first element as path to the video of the unknown data and second element as label.
    Default = None (add no ad to database)

   Notes:
    - The function requires files "model.h5", "AdNet.h5" and directory "Database". thus these must be in the same folder as of this file.
      otherwise the code will break 
  
  '''
  if unknown_data != None:
    if len(unknown_data) != 2:
      sys.exit("The len of the parameter ' unknown_data' must be 2")
  
  fir, last = os.path.splitext(test_video)
  if last != '.avi':
    sys.exit("the video must be in format '.avi'")

  #Check whether Database and model exists in the current directory 
  Database = "Database"
  AdNet = 'AdNet.h5'
  model = 'model.h5'
  
  curr_dir = os.getcwd() #current working directory

  if model not in os.listdir(curr_dir):
    sys.exit('"Model Not found: Make sure "model.h5" is in the same directory of this file and is named properly. ')
  else:
    print("Model found")
    
    #Load model
    from keras.models import load_model
    model = load_model(os.path.join (curr_dir, model))

  if Database not in os.listdir(curr_dir):
    sys.exit('"Dataabse Not found: Make sure folder "Database" is in the same directory of this file and is named properly. ')
  else:
    print("Database found")
    Database = os.path.join (curr_dir, Database)
  

  flag = False
  if unknown_data != None:
    
    #Check whether brand exists
    for labels in os.listdir(Database):
      if labels == unknown_data[1]:
        print("Ad already exist")
        flag =True
        break

    if not flag:
      print("Ad not found database. Adding to it")
      

      
      unk = model.predict(load_video(unknown_data[0]))

      filname = Database + '/' + unknown_data[0]

      with open(filename, 'wb') as file:
  
        np.save(file, unk) #save feature vector to database

  predict = model.predict(load_video(test_video))
  cos_sim_list = []
  label_list =[]

  #Compare with database
  for labels in os.listdir(Database):

    path_fv = os.path.join(Database, labels)

    fir, last = os.path.splitext(labels)

    with open(path_fv, 'rb') as file:
      fv = np.load(file)

    #Cosine similarity
    cos_sim = np.dot(fv, predict)/(np.linalg.norm(fv)*np.linalg.norm(predict))

    if cos_sim > 0.8:
      print ("the label is ", fir, " with cosine similarity with ", cos_sim*100, "%")
      return [fir, cos_sim]
    
    else:
      cos_sim_list.append(cos_sim)
      label_list.append(fir)
  
  #chose brand with highest similarity
  max_value =max(cos_sim_list)
  label_value =label_list.index(max_value)

  print ("the label is ", label_value, " with cosine similarity with ", max_value*100, "%")

  return [label_value, max_value]

# **Output**

In [ ]:
unknown_data = ['path', 'label']
test_video = 'path'
_ = prediction(test_video, unknown_data)